In [1]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.1 MB/s eta 0:00:00


In [13]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import Trainer

In [3]:
poems = load_dataset("text", data_files="/content/lspd.txt")

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
poems = poems['train'].train_test_split(test_size=0.1)

In [56]:
checkpoint = "Sakonii/distilgpt2-nepali"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = poems.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/410 [00:00<?, ? examples/s]

In [57]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [88]:
training_args = TrainingArguments(
    output_dir="poem-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [84]:
model_ckpt = "Sakonii/distilgpt2-nepali"
model = AutoModelForCausalLM.from_pretrained(model_ckpt)

In [60]:
tokenized_datasets = tokenized_datasets.rename_column("text", "labels")
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3681
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 410
    })
})

In [61]:
tokenized_datasets = tokenized_datasets.map(lambda example: {"labels": example["input_ids"]})

Map:   0%|          | 0/410 [00:00<?, ? examples/s]

In [89]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [90]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.416033
2,0.490400,0.418769
3,0.320600,0.449474
4,0.237400,0.474255
5,0.181400,0.485150


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=2305, training_loss=0.2873157674992162, metrics={'train_runtime': 539.989, 'train_samples_per_second': 34.084, 'train_steps_per_second': 4.269, 'total_flos': 601145836830720.0, 'train_loss': 0.2873157674992162, 'epoch': 5.0})

In [91]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.52


In [92]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_size="left")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [94]:
prompt = "तिमी"
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(inputs, max_new_tokens=50, do_sample=True, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
tokenizer.batch_decode(outputs,skip_special_tokens=True)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['तिमी, तिम्रो मुख हेर न, न भन? तिम्रा मुख हेरेर म हेर्छु, मलाई हेरी, तिमी नै हेर न, हेर? तिमी नै हेर न! तिमी आँखा हेर न, मैले हेरीन, यो हे मेरी मुख हेर ']

😆